In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import distance
from collections import OrderedDict
import math
import copy
from collections import defaultdict

In [2]:
def createGraph(data_1, data_2, data_col, combined_col):

    # Creating graph and nodes from dataset

    G = nx.Graph()
    for k in range(len(data_col)):
        if data_1.columns[k] in combined_col:
            G.add_nodes_from(data_1[data_1.columns[k]], t = "combined_col")
        else:
            G.add_nodes_from(data_1[data_1.columns[k]], t = data_col[k])
        if data_2.columns[k] in combined_col:
            G.add_nodes_from(data_2[data_2.columns[k]], t = "combined_col")
        else:
            G.add_nodes_from(data_2[data_2.columns[k]], t = data_col[k])

    # Creating edges
    for i in range(len(data_1)):
        for k in combined_col:
            if k in data_1.columns:
                G.add_edge(data_1.id[i], data_1[k][i])

    for i in range(len(data_2)):
        for k in combined_col:
            if k in data_2.columns:
                G.add_edge(data_2.id[i], data_2[k][i])

    k_type = [i for i in data_col if i not in combined_col]
    if len(combined_col) > 0:
        k_type.append("combined_col")
    return G, k_type

# Importing data

path = '../Data/Amazon-GoogleProducts/'
data_1 = pd.read_csv(path + 'Amazon.csv', encoding = "latin")
data_2 = pd.read_csv(path + 'GoogleProducts.csv', encoding = "latin")

# Converting all the data to string
for col in data_1.columns:
    data_1[col] = data_1[col].apply(str)
for col in data_2.columns:
    data_2[col] = data_2[col].apply(str)

# Defining k_type and columns to be combined into one type
data_col = ["id", "title", "description"]
word = ["title", "name", "description"]

G, k_type = createGraph(data_1, data_2, data_col, word)
G.remove_node("nan")

In [3]:
def createJaccardSim(G, k_type):
    CN_sim = []
    V = []
    for i in k_type:
        # to maintain the order of nodes, so that nodes in similarity matrix can be mapped
        temp = {'nodes':[], 'name':{}, 'position':{}}
        
        # extract the node values
        nodes = [x for x,y in G.nodes(data=True) if y['t'] == i]
        temp['nodes'] = nodes
        for x in range(len(nodes)):
            temp['name'][x] = nodes[x]
            temp['position'][nodes[x]] = x
        V.append(temp)
        n_t = len(nodes)
        
        # Similarity between the t-type nodes
        sim_t = np.zeros((n_t, n_t))
        for x in range(n_t):
            for y in range(n_t):
                sim_t[x][y] = 1 - distance.jaccard(nodes[x], nodes[y])
        
        # CN_sim has similarity matrices for all the k-type nodes	
        CN_sim.append(sim_t)
    return CN_sim, V


CN_sim, V = createJaccardSim(G, k_type)

In [4]:

def createTheGttMatrix(G,V):
    Gtt = OrderedDict()
    for i in range(len(V)):
        for j in range(i+1,len(V)):
            subgraph = G.subgraph(V[i]['nodes'] + V[j]['nodes'])
            G_t_t_dash = np.zeros((len(V[i]['nodes']), len(V[j]['nodes'])))
            for edge in subgraph.edges():
                G_t_t_dash[V[i]['position'][edge[0]], V[j]['position'][edge[1]]] = 1
            
            Gtt[(i,j)]=G_t_t_dash
    return Gtt

Gtt = createTheGttMatrix(G,V)


In [5]:
def calcSuperNodes(V):
    number_super_nodes =[]
    for v in V:
        number_super_nodes.append(int(math.sqrt(len(v['name']))))
    
    return number_super_nodes


number_super_nodes = calcSuperNodes(V)


In [6]:
number_super_nodes

[67, 89]

In [7]:
def initializeMapping(sim, p):
    C = []
    for i in range(len(sim)):
        # C_t is a mapping from all the nodes of t-type to its t-type supernode
        C_t = np.zeros((sim[i].shape[0], p[i]))
        for j in range(sim[i].shape[0]):
            # Randomly maps the t-type node to a t-type supernode
            C_t[j]=np.random.dirichlet(np.ones(p[i]),size=1)[0]
            #C_t[j, np.random.randint(low = 0, high = p[i])] = 1
        C.append(C_t)
    return C

C = initializeMapping(CN_sim, number_super_nodes)


In [8]:
len(set(np.argmax(C[0], axis=1)))

67

In [9]:
import copy
from collections import defaultdict

def createSummaryGraph(G, C, V, k_type):
    SG = copy.deepcopy(G)
    S = []
    
    for i in range(len(C)):
        S_t = {'super_nodes':defaultdict(list), 'name':{}, 'position':{}}
        for j in range(C[i].shape[0]):
            ind = list(C[i][j]).index(max(C[i][j]))
            S_t['super_nodes'][ind].append(V[i]['name'][j])
        
        for k,v in S_t['super_nodes'].items():
            S_t['name'][k]=v[0]
            S_t['position'][v[0]]=k
            for j in range(1,len(v)):
                SG = nx.contracted_nodes(SG, v[0], v[j])
                
        S.append(S_t)
            
    return SG, S

SG, S = createSummaryGraph(G, C, V, k_type)


In [10]:
def createTheSuperLinkMatrix(S):
    L = OrderedDict()
    for i in range(len(S)):
        S_t = S[i]
        for j in range(i+1,len(S)):
            S_t_dash = S[j]
            L_t_t_dash = {i:list(S_t['position'].keys()),
                          j:list(S_t_dash['position'].keys()),
                          'adj_matrix':np.zeros((len(S_t['super_nodes']), len(S_t_dash['super_nodes'])))}
            for k in range(0, len(S_t['super_nodes'])):
                L_t_t_dash['adj_matrix'][k]=np.random.dirichlet(np.ones(len(S_t_dash['super_nodes'])),size=1)[0]
            
            L[(i,j)] = L_t_t_dash
            
    return L


L = createTheSuperLinkMatrix(S)


In [11]:
C_backup = copy.deepcopy(C)
L_backup = copy.deepcopy(L)
S_backup = copy.deepcopy(S)

In [18]:
C = C_backup
L = L_backup
S = S_backup

In [12]:
def calcDiagonalMatrix(sim):
    D=[]
    for i in range(0,len(sim)):
        D.append(np.diag(np.sum(sim[i],axis=1)))
    
    return D

D = calcDiagonalMatrix(CN_sim)

In [13]:
def updateC(G, V, C, L, sim, D):
    for i in range(len(C)):
        C_t = C[i]
        D_t = D[i]
        sum1 = np.zeros(C_t.shape)
        sum2 = np.zeros(C_t.shape)
        for j in range(i+1, len(C)):
            C_t_dash = C[j]
            G_t_t_dash = Gtt[(i,j)]
            L_t_t_dash = L[(i,j)]['adj_matrix']
            temp1 = np.dot(C_t_dash, L_t_t_dash.transpose())
            sum1 += np.dot(G_t_t_dash, temp1)
            sum2 += np.dot(C_t, np.dot(L_t_t_dash, np.dot(C_t_dash.transpose(), temp1)))
        sum1 += np.matmul(sim[i], C_t)
        sum2 += np.matmul(D_t, C_t)
        if i >=1:
            C[i] = np.multiply(C_t, np.sqrt(np.divide(sum1, sum2)))
        for j in range(0,len(C[i])):
            C[i][j] = C[i][j]/sum(C[i][j])
    return C

def updateL(G, V, C, L):
    for i in range(len(C)):
        C_t = C[i]
        for j in range(i+1, len(C)):
            L_t_t_dash = L[(i,j)]['adj_matrix']
            C_t_dash = C[j]
            G_t_t_dash = Gtt[(i,j)]
            temp1 = np.dot(C_t.transpose(), np.dot(G_t_t_dash, C_t_dash))
            temp2 = np.dot(L_t_t_dash, np.dot(C_t_dash.transpose(), C_t_dash))
            temp3 = np.dot(C_t.transpose(), np.dot(C_t, temp2))
            L[(i,j)]['adj_matrix'] = np.multiply(L_t_t_dash, np.sqrt(np.divide(temp1, temp3)))
            for k in range(0,len(L[(i,j)]['adj_matrix'])):
                L[(i,j)]['adj_matrix'][k] = L[(i,j)]['adj_matrix'][k]/sum(L[(i,j)]['adj_matrix'][k])
    return L

In [14]:
def computeObjectiveFunction(C, L, V, CN_sim, G):
    k_type = len(C)
    first_term, second_term = 0, 0
    for type_ in range(k_type):
        sim_t = CN_sim[type_]
        C_t = C[type_]
        n_t = len(sim_t)#n_t: number of vertices in the type

        for i in range(n_t):
            for j in range(i):
                first_term += sim_t[i][j]*np.sum((C_t[i] - C_t[j])**2)

    for t in range(k_type):
        for t_dash in range(t+1,k_type):
            G_t_t_dash = Gtt[(t,t_dash)]
            temp1 = np.matmul(L[(t, t_dash)]['adj_matrix'],C[t_dash].transpose())
            temp2 = np.matmul(C[t], temp1)

            second_term += np.sum((G_t_t_dash - temp2)**2)

    objective = first_term + second_term
    return objective

In [15]:
## pickle CN_sim and V
import pickle

# Store data (serialize)
with open('sim.pickle', 'wb') as handle:
    pickle.dump(CN_sim, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('vertex.pickle', 'wb') as handle:
    pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('gtt.pickle', 'wb') as handle:
    pickle.dump(Gtt, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
# with open('filename.pickle', 'rb') as handle:
#     unserialized_data = pickle.load(handle)

In [19]:
import numpy as np
import copy

def computeTheta(C_t):
    vertex_cluster_contribution_sum = np.sum(C_t, axis = 0)
    theta = min(vertex_cluster_contribution_sum)
    return theta, vertex_cluster_contribution_sum

def computeObjectiveFunction(C, L, V, CN_sim, G):
    k_type = len(C)
    first_term, second_term = 0, 0
    for type_ in range(k_type):
        sim_t = CN_sim[type_]
        C_t = C[type_]
        n_t = len(sim_t)#n_t: number of vertices in the type

        for i in range(n_t):
            for j in range(i):
                first_term += sim_t[i][j]*np.sum((C_t[i] - C_t[j])**2)

    for t in range(k_type):
        for t_dash in range(t+1,k_type):
            G_t_t_dash = Gtt[(t,t_dash)]
            temp1 = np.matmul(L[(t, t_dash)]['adj_matrix'],C[t_dash].transpose())
            temp2 = np.matmul(C[t], temp1)

            second_term += np.sum((G_t_t_dash - temp2)**2)

    objective = first_term + second_term
    return objective

def removeSuperNodeFromL(L, t_type, supernode, index):
    L_temp = copy.deepcopy(L)
    for types, superlinks in L_temp.items():
        if types[0] > t_type:
            break
        elif types[0] == t_type:
            superlinks['adj_matrix'] = np.delete(superlinks['adj_matrix'], index , 0)
            #superlinks[types[0]].remove(supernode)
        
        elif types[1] == t_type:
            superlinks['adj_matrix'] = np.delete(superlinks['adj_matrix'], index , 1)
            #superlinks[types[1]].remove(supernode)

    return L_temp

def getOptimalSuperNode(G, SG, S, C, L, V, CN_sim):
    VC = []
    for i in range(0,len(C)):
        theta, vertex_cluster_contribution_sum = computeTheta(C[i])
        #print("theta:",theta)
        #VC.append(vertex_cluster_contribution_sum)
        index = np.argmin(vertex_cluster_contribution_sum)
        name_node = S[i]['name'][index]
        C[i] = np.delete(C[i],index,1)
        L = removeSuperNodeFromL(L,i,name_node,index)
    
    start_time = time.time()
    #SG, S = createSummaryGraph(G, C, V, k_type)
    print("Time taken to update the summary graph:", (time.time()-start_time)/60)
    return C, L

In [20]:
import time

for i in range(5):
        # Step 2: Call Search(G,S(G))
        C, L = getOptimalSuperNode(G, SG, S, C, L, V, CN_sim)
        
        # Step 3: update C and L
        C_values=[]
        change_C=[]
        L_values=[]
        change_L=[]
        for j in range(1):
            C_old = copy.deepcopy(C)
            C = updateC(G, V, C, L, CN_sim, D)
            L_old = copy.deepcopy(L)
            L = updateL(G, V, C, L)
            sum_C=0
            sum_L=0
            for i in range(0,len(C)):
                sum_C+=np.sum(abs(C[i]-C_old[i]))
            for tt_dash in L:
                sum_L+=np.sum(abs(L[tt_dash]['adj_matrix']-L_old[tt_dash]['adj_matrix']))
            
            print(sum_C,sum_L)
            change_C.append(sum_C)
            change_L.append(sum_L)
            C_values.append(C)
            L_values.append(L)
        
        index = change_C.index(min(change_C))
        C = C_values[index]
        index = change_L.index(min(change_L))
        L = L_values[index]
        
        # Construct the new summary graph S(G)
        indices_to_keep = []
        t = list(set(np.argmax(C[0], axis=1)))
        indices_to_keep.append(t)
        C[0] = C[0][:,t]
        t = list(set(np.argmax(C[1], axis=1)))
        indices_to_keep.append(t)
        C[1] = C[1][:,t]
        #update the Ltt
        #hard-coding again -----
        temp_L = copy.deepcopy(L[(0,1)]['adj_matrix'])
        temp_L = temp_L[indices_to_keep[0],:]
        temp_L = temp_L[:,indices_to_keep[1]]
        
        L[(0,1)]['adj_matrix'] = temp_L
        start_time = time.time()
        SG, S = createSummaryGraph(G, C, V, k_type)
        print("Time taken to update the Summary Graph:", (time.time()-start_time)/60)
        print("Super Nodes remaining: ", len(indices_to_keep[0]), len(indices_to_keep[1]))
        # Calculate the new objective function
        start_time = time.time()
        final_objective = computeObjectiveFunction(C, L, V, CN_sim, G)
        print("Time taken to calculate the objective function:",(time.time()-start_time)/60)
        
#         if final_objective<initial_objective:
#             SG_final = copy.deepcopy(SG)
#             C_final = copy.deepcopy(C)
#             S_final = copy.deepcopy(S)
#             L_final = copy.deepcopy(L)
#             initial_objective = final_objective

Time taken to update the summary graph: 1.5894571940104166e-08
2762.7663220598474 0.7159092596283616
Time taken to update the Summary Graph: 8.502154036362965
Super Nodes remaining:  66 88
Time taken to calculate the objective function: 4.929713102181752
Time taken to update the summary graph: 0.0
1645.3109661475683 0.8312959515498803
Time taken to update the Summary Graph: 8.690619130929312
Super Nodes remaining:  65 87
Time taken to calculate the objective function: 6.260622084140778
Time taken to update the summary graph: 1.9868214925130207e-08
935.2596847817188 0.829565095029573
Time taken to update the Summary Graph: 8.666890414555867
Super Nodes remaining:  64 86
Time taken to calculate the objective function: 4.927519047260285
Time taken to update the summary graph: 0.0
526.3652487697968 0.6210521312110531
Time taken to update the Summary Graph: 8.59345358212789
Super Nodes remaining:  63 85
Time taken to calculate the objective function: 4.941975772380829
Time taken to update t

In [21]:
l = [1,2,3]

def temp_func(l):
    l[1] = "lisa"

In [ ]:
t = list(set(np.argmax(C[1], axis=1)))

In [ ]:
a = np.array([[1,2,3],[2,3,4],[4,5,6]])
x = [0,1]
y = [0,1]

a[x,y]

In [ ]:
len(set(np.argmax(C[1], axis=1)))

In [ ]:
number_super_nodes

In [ ]:
SG